# Testing

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import keras

from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical

In [2]:
with open("kddcup.names",'r') as f:
    print(f.read())

back,buffer_overflow,ftp_write,guess_passwd,imap,ipsweep,land,loadmodule,multihop,neptune,nmap,normal,perl,phf,pod,portsweep,rootkit,satan,smurf,spy,teardrop,warezclient,warezmaster.
duration: continuous.
protocol_type: symbolic.
service: symbolic.
flag: symbolic.
src_bytes: continuous.
dst_bytes: continuous.
land: symbolic.
wrong_fragment: continuous.
urgent: continuous.
hot: continuous.
num_failed_logins: continuous.
logged_in: symbolic.
num_compromised: continuous.
root_shell: continuous.
su_attempted: continuous.
num_root: continuous.
num_file_creations: continuous.
num_shells: continuous.
num_access_files: continuous.
num_outbound_cmds: continuous.
is_host_login: symbolic.
is_guest_login: symbolic.
count: continuous.
srv_count: continuous.
serror_rate: continuous.
srv_serror_rate: continuous.
rerror_rate: continuous.
srv_rerror_rate: continuous.
same_srv_rate: continuous.
diff_srv_rate: continuous.
srv_diff_host_rate: continuous.
dst_host_count: continuous.
dst_host_srv_count: con

In [3]:
cols="""duration,
protocol_type,
service,
flag,
src_bytes,
dst_bytes,
land,
wrong_fragment,
urgent,
hot,
num_failed_logins,
logged_in,
num_compromised,
root_shell,
su_attempted,
num_root,
num_file_creations,
num_shells,
num_access_files,
num_outbound_cmds,
is_host_login,
is_guest_login,
count,
srv_count,
serror_rate,
srv_serror_rate,
rerror_rate,
srv_rerror_rate,
same_srv_rate,
diff_srv_rate,
srv_diff_host_rate,
dst_host_count,
dst_host_srv_count,
dst_host_same_srv_rate,
dst_host_diff_srv_rate,
dst_host_same_src_port_rate,
dst_host_srv_diff_host_rate,
dst_host_serror_rate,
dst_host_srv_serror_rate,
dst_host_rerror_rate,
dst_host_srv_rerror_rate"""

columns=[]
for c in cols.split(','):
    if(c.strip()):
       columns.append(c.strip())

columns.append('target')
#print(columns)
print(len(columns))

42


In [4]:
attacks_types = {
    'normal.': 'normal',
'back.': 'dos',
'buffer_overflow.': 'u2r',
'ftp_write.': 'r2l',
'guess_passwd.': 'r2l',
'imap.': 'r2l',
'ipsweep.': 'probe',
'land.': 'dos',
'loadmodule.': 'u2r',
'multihop.': 'r2l',
'neptune.': 'dos',
'nmap.': 'probe',
'perl.': 'u2r',
'phf.': 'r2l',
'pod.': 'dos',
'portsweep.': 'probe',
'rootkit.': 'u2r',
'satan.': 'probe',
'smurf.': 'dos',
'spy.': 'r2l',
'teardrop.': 'dos',
'warezclient.': 'r2l',
'warezmaster.': 'r2l',
}
attacks_types

{'normal.': 'normal',
 'back.': 'dos',
 'buffer_overflow.': 'u2r',
 'ftp_write.': 'r2l',
 'guess_passwd.': 'r2l',
 'imap.': 'r2l',
 'ipsweep.': 'probe',
 'land.': 'dos',
 'loadmodule.': 'u2r',
 'multihop.': 'r2l',
 'neptune.': 'dos',
 'nmap.': 'probe',
 'perl.': 'u2r',
 'phf.': 'r2l',
 'pod.': 'dos',
 'portsweep.': 'probe',
 'rootkit.': 'u2r',
 'satan.': 'probe',
 'smurf.': 'dos',
 'spy.': 'r2l',
 'teardrop.': 'dos',
 'warezclient.': 'r2l',
 'warezmaster.': 'r2l'}

In [5]:
path = "corrected"
df = pd.read_csv(path,names=columns)

In [6]:
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,target
0,0,udp,private,SF,105,146,0,0,0,0,...,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,normal.
1,0,udp,private,SF,105,146,0,0,0,0,...,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,normal.
2,0,udp,private,SF,105,146,0,0,0,0,...,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,normal.
3,0,udp,private,SF,105,146,0,0,0,0,...,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,snmpgetattack.
4,0,udp,private,SF,105,146,0,0,0,0,...,254,1.0,0.01,0.01,0.0,0.0,0.0,0.0,0.0,snmpgetattack.


In [7]:
df.shape

(311029, 42)

## Data preprocessing

In [8]:
df = df[df.target.isin(list(attacks_types.keys()))]

In [9]:
df['attack_type'] = df.target.apply(lambda r:attacks_types[r[:]])

In [10]:
df.attack_type.unique()

array(['normal', 'dos', 'probe', 'r2l', 'u2r'], dtype=object)

In [11]:
df.isnull().sum()

duration                       0
protocol_type                  0
service                        0
flag                           0
src_bytes                      0
dst_bytes                      0
land                           0
wrong_fragment                 0
urgent                         0
hot                            0
num_failed_logins              0
logged_in                      0
num_compromised                0
root_shell                     0
su_attempted                   0
num_root                       0
num_file_creations             0
num_shells                     0
num_access_files               0
num_outbound_cmds              0
is_host_login                  0
is_guest_login                 0
count                          0
srv_count                      0
serror_rate                    0
srv_serror_rate                0
rerror_rate                    0
srv_rerror_rate                0
same_srv_rate                  0
diff_srv_rate                  0
srv_diff_h

In [12]:
#Finding categorical features
num_cols = df._get_numeric_data().columns

cate_cols = list(set(df.columns)-set(num_cols))
cate_cols.remove('target')
cate_cols.remove('attack_type')

cate_cols

['flag', 'protocol_type', 'service']

In [13]:
df = df.dropna('columns')# drop columns with NaN

df = df[[col for col in df if df[col].nunique() > 1]]# keep columns where there are more than 1 unique values

In [14]:
df.drop('num_root',axis = 1,inplace = True)
df.drop('srv_serror_rate',axis = 1,inplace = True)
df.drop('srv_rerror_rate',axis = 1, inplace=True)
df.drop('dst_host_srv_serror_rate',axis = 1, inplace=True)
df.drop('dst_host_serror_rate',axis = 1, inplace=True)
df.drop('dst_host_rerror_rate',axis = 1, inplace=True)
df.drop('dst_host_srv_rerror_rate',axis = 1, inplace=True)
df.drop('dst_host_same_srv_rate',axis = 1, inplace=True)

In [15]:
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,target,attack_type
0,0,udp,private,SF,105,146,0,0,0,0,...,1.0,0.0,0.0,255,254,0.01,0.0,0.0,normal.,normal
1,0,udp,private,SF,105,146,0,0,0,0,...,1.0,0.0,0.0,255,254,0.01,0.0,0.0,normal.,normal
2,0,udp,private,SF,105,146,0,0,0,0,...,1.0,0.0,0.0,255,254,0.01,0.0,0.0,normal.,normal
6,0,udp,domain_u,SF,29,0,0,0,0,0,...,0.5,1.0,0.0,10,3,0.30,0.3,0.0,normal.,normal
7,0,udp,private,SF,105,146,0,0,0,0,...,1.0,0.0,0.0,255,253,0.01,0.0,0.0,normal.,normal


In [16]:
df.shape

(292300, 34)

In [17]:
df.columns

Index(['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
       'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
       'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
       'su_attempted', 'num_file_creations', 'num_shells', 'num_access_files',
       'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate',
       'rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate',
       'dst_host_count', 'dst_host_srv_count', 'dst_host_diff_srv_rate',
       'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'target',
       'attack_type'],
      dtype='object')

# FEATURE MAPPING

In [18]:
df['protocol_type'].value_counts()

icmp    164864
tcp     110906
udp      16530
Name: protocol_type, dtype: int64

In [19]:
#protocol_type feature mapping
pmap = {'icmp':0,'tcp':1,'udp':2}
df['protocol_type'] = df['protocol_type'].map(pmap)

In [20]:
df['flag'].value_counts()

SF        232381
REJ        40892
S0         17523
RSTO        1012
RSTR         357
SH            84
S1            22
S2            15
S3             8
OTH            4
RSTOS0         2
Name: flag, dtype: int64

In [21]:
#flag feature mapping
fmap = {'SF':0,'S0':1,'REJ':2,'RSTR':3,'RSTO':4,'SH':5 ,'S1':6 ,'S2':7,'RSTOS0':8,'S3':9 ,'OTH':10}
df['flag'] = df['flag'].map(fmap)

In [22]:
attackmap = {'dos':0, 'normal':1, 'probe':2, 'r2l':3, 'u2r':4}
df['attack_type'] = df['attack_type'].map(attackmap)

In [23]:
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,target,attack_type
0,0,2,private,0,105,146,0,0,0,0,...,1.0,0.0,0.0,255,254,0.01,0.0,0.0,normal.,1
1,0,2,private,0,105,146,0,0,0,0,...,1.0,0.0,0.0,255,254,0.01,0.0,0.0,normal.,1
2,0,2,private,0,105,146,0,0,0,0,...,1.0,0.0,0.0,255,254,0.01,0.0,0.0,normal.,1
6,0,2,domain_u,0,29,0,0,0,0,0,...,0.5,1.0,0.0,10,3,0.30,0.3,0.0,normal.,1
7,0,2,private,0,105,146,0,0,0,0,...,1.0,0.0,0.0,255,253,0.01,0.0,0.0,normal.,1


In [24]:
df.drop('service',axis = 1,inplace= True)
df.drop('target', axis = 1, inplace=True)

In [25]:
df.shape

(292300, 32)

In [26]:
df.head()

,duration,protocol_type,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,...,rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,attack_type
0,0,2,0,105,146,0,0,0,0,0,...,0.0,1.0,0.0,0.0,255,254,0.01,0.0,0.0,1
1,0,2,0,105,146,0,0,0,0,0,...,0.0,1.0,0.0,0.0,255,254,0.01,0.0,0.0,1
2,0,2,0,105,146,0,0,0,0,0,...,0.0,1.0,0.0,0.0,255,254,0.01,0.0,0.0,1
6,0,2,0,29,0,0,0,0,0,0,...,0.0,0.5,1.0,0.0,10,3,0.30,0.3,0.0,1
7,0,2,0,105,146,0,0,0,0,0,...,0.0,1.0,0.0,0.0,255,253,0.01,0.0,0.0,1


In [27]:
df.dtypes

duration                         int64
protocol_type                    int64
flag                             int64
src_bytes                        int64
dst_bytes                        int64
land                             int64
wrong_fragment                   int64
urgent                           int64
hot                              int64
num_failed_logins                int64
logged_in                        int64
num_compromised                  int64
root_shell                       int64
su_attempted                     int64
num_file_creations               int64
num_shells                       int64
num_access_files                 int64
is_host_login                    int64
is_guest_login                   int64
count                            int64
srv_count                        int64
serror_rate                    float64
rerror_rate                    float64
same_srv_rate                  float64
diff_srv_rate                  float64
srv_diff_host_rate       

In [28]:
y_test = df['attack_type']
x_test = df.drop('attack_type', axis=1)

In [29]:
y_test.shape

(292300,)

In [30]:
x_test.shape

(292300, 31)

In [31]:
sc = MinMaxScaler()
x_test = sc.fit_transform(x_test)

In [32]:
model = load_model('ann.h5')

In [33]:
y_pred = model.predict(x_test)

In [34]:
y_res = y_pred.argmax(axis=1)

In [35]:
print("Test accuracy: %.2f%%"%(100*accuracy_score(y_test, y_res)))

Test accuracy: 97.69%


## Remapping output values to categories

In [36]:
reverse_attackmap = {k:v for v,k in attackmap.items()}

In [39]:
results = pd.DataFrame(y_res)

In [47]:
results = results[0].map(reverse_attackmap)

In [51]:
results

0         normal
1         normal
2         normal
3         normal
4         normal
           ...  
292295    normal
292296    normal
292297    normal
292298    normal
292299    normal
Name: 0, Length: 292300, dtype: object